In [1]:
import nltk

In [2]:
from nltk.corpus import words

In [4]:
nltk.download("words")

[nltk_data] Downloading package words to /home/cyber2/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
words.words()

### Trie data structure for faster retrieval

In [80]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word[::-1]:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.end_of_word = True

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.end_of_word

### If word in trie, append, else use min edit distance for nearest word

In [81]:
def spell_check(sentence, trie):
    corrected_sentence = []
    for word in sentence.split():
        if trie.search(word):
            corrected_sentence.append(word)
        else:
            suggestions = []
            for char in trie.root.children:
                node = trie.root.children[char]
                if char == word[0]:
                    for i in range(1, len(word)):
                        if word[i] not in node.children:
                            break
                        node = node.children[word[i]]
                    else:
                        if node.end_of_word:
                            suggestions.append(char + word[1:])
                        for child in node.children:
                            suggestions.append(char + child + word[2:])
            if suggestions:
                corrected_sentence.append(min(suggestions, key=lambda x: edit_distance(x, word)))
            else:
                corrected_sentence.append(word)
    return ' '.join(corrected_sentence)

In [82]:
def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])
    return dp[m][n]

In [84]:
trie = Trie()
for i in words.words():
    trie.insert(i)

### This just checks for prefix and we need a language model to get proper replacement for word

In [85]:
print(spell_check('During the summer we have the best ueather.', trie))
print(spell_check('I have a black ueather jacket, so nice.', trie))

During the summer wr have the best ueather.
I have a black ueather jacket, so nice.
